In [2]:
import pandas as pd 

df = pd.read_csv("symtab.csv")
df.head()

,Scope,Type,Variable Name,Memloc,Line Numbers
0,global,void,main,4,5
1,global,int,gcd,1,1 3 8
2,gcd,int,u,2,1 2 3 3
3,gcd,int,v,3,1 2 3 3 3
4,main,int,x,5,6 7 8


In [3]:
def getMemLoc(row):
    if "global" in row["Scope"]:
        return row["Memloc"]
    elif "main" in row["Scope"]:
        return row["Memloc"] - df.loc[df["Variable Name"] == row["Scope"],"Memloc"].values[0]
    else:
        return row["Memloc"] - df.loc[df["Variable Name"] == row["Scope"],"Memloc"].values[0] + 1

df["Scope Memloc"] = df.apply(getMemLoc, axis=1)
df.sort_values(by='Memloc')


,Scope,Type,Variable Name,Memloc,Line Numbers,Scope Memloc
1,global,int,gcd,1,1 3 8,1
2,gcd,int,u,2,1 2 3 3,2
3,gcd,int,v,3,1 2 3 3 3,3
0,global,void,main,4,5,4
4,main,int,x,5,6 7 8,1
5,main,int,y,6,6 7 8,2


In [65]:
import re
reg = [None]*25
scope = 'global'
params_reg = []

def deallocateReg(f):
    global reg, params_reg

    index = set()

    for r in params_reg:
        index.add(int(r))

    pos = f.tell()
    nextLine = f.readline().strip()
    while nextLine != "":
        if "$t" in nextLine: 
            for i, r in enumerate(reg):
                if r is not None and re.search(r"{}(?![0-9])".format(re.escape(r)), nextLine) and i not in index: index.add(i)

        nextLine = f.readline().strip()

    for i in range(len(reg)):
        if i not in index: reg[i] = None

    f.seek(pos)


with open("test.asm", "w") as asm:

    with open("../test.ic", "r") as f:

        asm.write("J main\n")
        line = f.readline().strip()
        while line != '':
            quad = line.split(" ")

            for i in quad:
                if "$t" in i: 
                    aux = i.split("(")[1][:-1] if "(" in i else i
                    if not aux in reg: reg[reg.index(None)]=aux

            match quad[0]:
                case "INPUT":
                    asm.write(f"IN $t{reg.index(quad[1])}\n")

                case "OUTPUT":
                    asm.write(f"OUT $t{reg.index(quad[1])}\n")
                    # params_reg.pop(0)

                case "FUN":
                    scope = quad[2]
                    asm.write(f".{quad[2]}\n")

                    if quad[2] != "main":
                        asm.write("SW $r31 $r29 1\n")
                        asm.write("ADDI $r30 $r30 1\n")

                case "RET":
                    asm.write(f"ADDI $r28 $t{reg.index(quad[1])} 0\n")

                case "ALLOC":
                    asm.write("ADDI $r30 $r30 1\n")

                case "ARG":
                    asm.write("ADDI $r30 $r30 1\n")

                case "PARAM":
                    params_reg.insert(0, f"{reg.index(quad[1])}")

                case "CALL":

                    ## Empilha
                    saveds_reg = [None]*25
                    for i, r in enumerate(reg.copy()):
                        if r is not None:
                            asm.write(f"SW $t{i} $r30 0\n")
                            asm.write(f"ADDI $r30 $r30 1\n")
                            saveds_reg[i] = r
                        
                    reg = [None]*25

                    asm.write("SW $r29 $r30 0\n")
                    asm.write("ADDI $r29 $r30 0\n")
                    asm.write("ADDI $r30 $r30 1\n")

                    ## Call
                    asm.write(f"JAL {quad[2]}\n")
                    params_reg = []

                    ## Desempilha
                    reg = saveds_reg
                    for i, r in enumerate(reversed(reg.copy())):
                        index = len(reg) - 1 - i
                        if r is not None:
                            asm.write(f"SUBI $r30 $r30 1\n")
                            asm.write(f"LW $t{index} $r30 0\n")


                case "LOAD":
                    scopevar = scope
                    var = quad[2]
                    if not df.loc[df['Scope'] == scopevar, 'Variable Name'].isin([var]).any(): scopevar = 'global' 

                    if "(" in var: 
                        asm.write(f"Unknown {var}\n")
                    #     addr = quad[2].split('(')
                    #     var = addr[0]
                    #     offset = addr[1].strip(")")
                    # else: 
                    #     offset = 0
                    #     var = quad[2]

                    else: asm.write(f"LW $t{reg.index(quad[1])} $r29 {df.loc[((df['Scope'] == scopevar) & (df['Variable Name'] == var)), 'Scope Memloc'].values[0]}\n")
                
                case "STORE":
                    scopevar = scope
                    var = quad[2]
                    if not df.loc[df['Scope'] == scopevar, 'Variable Name'].isin([var]).any(): scopevar = 'global' 

                    if "(" in var: 
                        asm.write(f"Unknown {var}\n")
                    #     addr = quad[2].split('(')
                    #     var = addr[0]
                    #     offset = addr[1].strip(")")
                    # else: 
                    #     offset = 0
                    #     var = quad[2]

                    else: asm.write(f"SW $t{reg.index(quad[1])} $r29 {df.loc[((df['Scope'] == scopevar) & (df['Variable Name'] == var)), 'Scope Memloc'].values[0]}\n")

                case "LAB":
                    asm.write(f".{quad[1]}\n")

                case "MAIS":
                    asm.write(f"ADD $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
                
                case "ADDI":
                    if quad[2] == "$zero": asm.write(f"ADDI $t{reg.index(quad[1])} {quad[2]} {quad[3]}\n")
                    else: asm.write(f"ADDI $t{reg.index(quad[1])} $t{reg.index(quad[2])} {quad[3]}\n")

                case "MENOS":
                    asm.write(f"SUB $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "VEZES":
                    asm.write(f"MULT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
                
                case "SOBRE":
                    asm.write(f"DIV $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "MENORQ":
                    asm.write(f"LT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "MENORI":
                    asm.write(f"LE $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "MAIORQ":
                    asm.write(f"GT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "MAIORI":
                    asm.write(f"GE $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

                case "IGUAL":
                    asm.write(f"EQ $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
                
                case "DIF":
                    asm.write(f"DF $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
                
                case "IFF":
                    asm.write(f"IFF $t{reg.index(quad[1])} {quad[2]}\n")

                case "GOTO":
                    asm.write(f"J {quad[1]}\n")

                case "END":
                    asm.write(f".END {quad[1]}\n")
                    asm.write("LW $r31 $r29 1\n")
                    asm.write(f"JR $r31\n")

                case "HALT":
                    asm.write(f"HALT\n")

                case _: asm.write(f"Unknown - {quad[0]}\n")

            print(reg)
            deallocateReg(f)
            line = f.readline().strip()


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['$t0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['$t0', '$t1', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['$t0', '$t1', '$t2', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, '$t2', None, None, None, None, None, None, None, None, None, None, None, No